# Manutenzione preventiva - DoS Attacks

In [58]:
import pandas as pd
import json as json
import ipaddress as ip 
import numpy as np
import warnings
import functions_notebook_new as fn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC


warnings.simplefilter(action='ignore', category=FutureWarning)

In [60]:
### costanti
# clean captures
clean_capture = 'captures\\captures1_v2\\clean\\eth2dump-clean-0,5h_1.json'

# ping flooding captures 1 min
pf_capture_1 = 'captures\\captures1_v2\\mitm\\eth2dump-mitm-change-15m-1h_1.json'
pf_capture_2 = 'captures\\captures1_v2\\mitm\\eth2dump-mitm-change-15m-1h_1.json'
pf_capture_3 = 'captures\\captures1_v2\\mitm\\eth2dump-mitm-change-15m-1h_1.json'

levels_of_interest = ['frame', 'eth', 'tcp', 'arp', 'udp', 'modbus', 'icmp', 'ip']

### variabili

In [61]:
### estrazione dati
# packets clean capture
packets_cc = fn.load_packets_from_file(clean_capture)

# packets attack captures (es. packets ping flooding attack 1m per cattura di 0.5 = pkt_pf_1m_05h)
pkt_pf_1m_05h = fn.load_packets_from_file(pf_capture_1)
pkt_pf_5m_05h = fn.load_packets_from_file(pf_capture_2)
pkt_pf_15m_05h = fn.load_packets_from_file(pf_capture_3)

In [62]:
### features
# clean features
features_cc = fn.extract_features_from_packets(packets_cc, levels_of_interest)

# ping flooding attack features
features_pf_1m_05h = fn.extract_features_from_packets(pkt_pf_1m_05h, levels_of_interest)
features_pf_5m_05h = fn.extract_features_from_packets(pkt_pf_5m_05h, levels_of_interest)
features_pf_15m_05h =fn.extract_features_from_packets(pkt_pf_15m_05h, levels_of_interest)

In [50]:
### Dataframes
# df clean capture
df_clean = pd.DataFrame(features_cc)

# df ping flooding attacks captures
df_pf_1m_05h = pd.DataFrame(features_pf_1m_05h)
df_pf_5m_05h = pd.DataFrame(features_pf_5m_05h)
df_pf_15m_05h = pd.DataFrame(features_pf_15m_05h)



df_pf_1m_05h['label']=1
df_pf_5m_05h['label']=1
df_pf_15m_05h['label']=1
df_clean['label']=0
pd_concatenation = pd.concat([df_clean,df_pf_1m_05h, df_pf_5m_05h, df_pf_15m_05h])

In [51]:
df_colonne_senza_nan = pd_concatenation.loc[:, pd_concatenation.isna().sum() == 0].copy()


In [52]:
len(df_colonne_senza_nan.select_dtypes(include=['object']).columns)

60

In [53]:
print(df_colonne_senza_nan['label'])

0        0
1        0
2        0
3        0
4        0
        ..
29687    1
29688    1
29689    1
29690    1
29691    1
Name: label, Length: 161226, dtype: int64


In [54]:
df_colonne_senza_nan.select_dtypes(include=['object']).iloc[:10,:10]

,frame.frame.encap_type,frame.frame.time,frame.frame.time_utc,frame.frame.time_epoch,frame.frame.offset_shift,frame.frame.time_delta,frame.frame.time_delta_displayed,frame.frame.time_relative,frame.frame.number,frame.frame.len
0,1,"Sep 8, 2018 22:54:30.055515000 ora legale Eur...","Sep 8, 2018 20:54:30.055515000 UTC",1536440070.055515000,0.000000000,0.000000000,0.000000000,0.000000000,1,64
1,1,"Sep 8, 2018 22:54:30.111272000 ora legale Eur...","Sep 8, 2018 20:54:30.111272000 UTC",1536440070.111272000,0.000000000,0.055757000,0.055757000,0.055757000,2,60
2,1,"Sep 8, 2018 22:54:30.205937000 ora legale Eur...","Sep 8, 2018 20:54:30.205937000 UTC",1536440070.205937000,0.000000000,0.094665000,0.094665000,0.150422000,3,66
3,1,"Sep 8, 2018 22:54:30.209039000 ora legale Eur...","Sep 8, 2018 20:54:30.209039000 UTC",1536440070.209039000,0.000000000,0.003102000,0.003102000,0.153524000,4,85
4,1,"Sep 8, 2018 22:54:30.423275000 ora legale Eur...","Sep 8, 2018 20:54:30.423275000 UTC",1536440070.423275000,0.000000000,0.214236000,0.214236000,0.367760000,5,60
5,1,"Sep 8, 2018 22:54:30.517529000 ora legale Eur...","Sep 8, 2018 20:54:30.517529000 UTC",1536440070.517529000,0.000000000,0.094254000,0.094254000,0.462014000,6,66
6,1,"Sep 8, 2018 22:54:30.528387000 ora legale Eur...","Sep 8, 2018 20:54:30.528387000 UTC",1536440070.528387000,0.000000000,0.010858000,0.010858000,0.472872000,7,85
7,1,"Sep 8, 2018 22:54:30.735268000 ora legale Eur...","Sep 8, 2018 20:54:30.735268000 UTC",1536440070.735268000,0.000000000,0.206881000,0.206881000,0.679753000,8,60
8,1,"Sep 8, 2018 22:54:30.751871000 ora legale Eur...","Sep 8, 2018 20:54:30.751871000 UTC",1536440070.751871000,0.000000000,0.016603000,0.016603000,0.696356000,9,82
9,1,"Sep 8, 2018 22:54:30.829737000 ora legale Eur...","Sep 8, 2018 20:54:30.829737000 UTC",1536440070.829737000,0.000000000,0.077866000,0.077866000,0.774222000,10,66


### Feature tipo intero
frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui	
frame.frame.cap_len	frame.frame.marked	frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]	
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]	eth.eth.dst_tree.eth.lg	eth.eth.dst_tree.eth.dst.ig_raw[0]	eth.eth.dst_tree.eth.dst.ig	eth.eth.dst_tree.eth.ig_raw[0]	eth.eth.dst_tree.eth.ig
eth.eth.src
eth.eth.src_tree.eth.src.oui	
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]	eth.eth.src_tree.eth.src.lg	eth.eth.src_tree.eth.lg_raw[0]	eth.eth.src_tree.eth.lg	eth.eth.src_tree.eth.src.ig_raw[0]	eth.eth.src_tree.eth.src.ig	eth.eth.src_tree.eth.ig_raw[0]	eth.eth.src_tree.eth.ig
#### ESADECIMALE
eth.eth.dst_raw[0]	 eth.eth.dst_tree.eth.dst_resolved_raw[0] eth.eth.dst_tree.eth.dst.oui_raw[0] eth.eth.dst_tree.eth.addr_raw[0] eth.eth.dst_tree.eth.addr.oui_raw[0]
eth.eth.src_raw[0]
eth.eth.src_tree.eth.src_resolved_raw[0]	
eth.eth.src_tree.eth.src.oui_raw[0]
eth.eth.src_tree.eth.src.oui_resolved_raw[0]
eth.eth.src_tree.eth.addr_raw[0]
eth.eth.src_tree.eth.addr
eth.eth.src_tree.eth.addr_resolved_raw[0]
eth.eth.src_tree.eth.addr.oui_raw[0]
eth.eth.src_tree.eth.addr.oui_resolved_raw[0]				
#### MAC ADDRESS	
eth.eth.dst mac address eth.eth.dst_tree.eth.addr eth.eth.dst_tree.eth.addr_resolved_raw[0] 	 (da rimuovere : e conversione in esadecimale)

### Feature tipo float
frame.frame.time_epoch
frame.frame.offset_shift
frame.frame.time_delta
frame.frame.time_delta_displayed
frame.frame.time_relative
### Feature tipo string
frame.frame.protocols	da ricavare i protocolli contenuti all'interno del frame
#### DA SCARTARE
'frame.frame.coloring_rule.name'
'frame.frame.coloring_rule.string'
'eth.eth.dst_tree.eth.dst_resolved'
'eth.eth.dst_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.src_resolved'
'eth.eth.src_tree.eth.src.oui_resolved'	
'eth.eth.src_tree.eth.addr_resolved'
'eth.eth.src_tree.eth.addr.oui_resolved'

In [55]:
pd_nona_bck = df_colonne_senza_nan.copy()
df_colonne_senza_nan.drop(columns=['frame.frame.coloring_rule.name','frame.frame.coloring_rule.string', 'eth.eth.dst_tree.eth.dst_resolved', 'eth.eth.dst_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.src_resolved', 'eth.eth.src_tree.eth.src.oui_resolved', 'eth.eth.src_tree.eth.addr_resolved', 'eth.eth.src_tree.eth.addr.oui_resolved'], inplace=True)
column_names = df_colonne_senza_nan.columns.tolist()
print(column_names)


['frame.frame.encap_type', 'frame.frame.time', 'frame.frame.time_utc', 'frame.frame.time_epoch', 'frame.frame.offset_shift', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'frame.frame.marked', 'frame.frame.ignored', 'frame.frame.protocols', 'eth.eth.dst_raw[0]', 'eth.eth.dst_raw[1]', 'eth.eth.dst_raw[2]', 'eth.eth.dst_raw[3]', 'eth.eth.dst_raw[4]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst_resolved_raw[1]', 'eth.eth.dst_tree.eth.dst_resolved_raw[2]', 'eth.eth.dst_tree.eth.dst_resolved_raw[3]', 'eth.eth.dst_tree.eth.dst_resolved_raw[4]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[1]', 'eth.eth.dst_tree.eth.dst.oui_raw[2]', 'eth.eth.dst_tree.eth.dst.oui_raw[3]', 'eth.eth.dst_tree.eth.dst.oui_raw[4]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr_raw[1]', 'eth.e

In [56]:
len(df_colonne_senza_nan)

161226

In [57]:
fn.df_convert(df_colonne_senza_nan)
df_colonne_senza_nan

frame.frame.encap_type
frame.frame.number
frame.frame.len
eth.eth.dst_tree.eth.dst.oui
frame.frame.cap_len
frame.frame.marked
frame.frame.ignored
eth.eth.dst_tree.eth.dst.lg_raw[0]
eth.eth.dst_tree.eth.addr.oui
eth.eth.dst_tree.eth.dst.lg
eth.eth.dst_tree.eth.lg_raw[0]
eth.eth.dst_tree.eth.lg
eth.eth.dst_tree.eth.dst.ig_raw[0]
eth.eth.dst_tree.eth.dst.ig
eth.eth.dst_tree.eth.ig_raw[0]
eth.eth.dst_tree.eth.ig
eth.eth.src_tree.eth.src.oui
eth.eth.src_tree.eth.addr.oui
eth.eth.src_tree.eth.src.lg_raw[0]
eth.eth.src_tree.eth.src.lg
eth.eth.src_tree.eth.lg_raw[0]
eth.eth.src_tree.eth.lg
eth.eth.src_tree.eth.src.ig_raw[0]
eth.eth.src_tree.eth.src.ig
eth.eth.src_tree.eth.ig_raw[0]
eth.eth.src_tree.eth.ig


AttributeError: Can only use .str accessor with string values!

In [31]:
print(df_nonan_converted.dtypes)

frame.frame.encap_type            object
frame.frame.time                  object
frame.frame.time_utc              object
frame.frame.time_epoch            object
frame.frame.offset_shift          object
                                   ...  
eth.eth.src_tree.eth.ig_raw[2]     int64
eth.eth.src_tree.eth.ig_raw[3]     int64
eth.eth.src_tree.eth.ig_raw[4]     int64
eth.eth.src_tree.eth.ig           object
label                              int64
Length: 141, dtype: object


In [15]:
print(df_nonan_converted.columns.tolist())

['frame.frame.encap_type', 'frame.frame.time', 'frame.frame.time_utc', 'frame.frame.time_epoch', 'frame.frame.offset_shift', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'frame.frame.marked', 'frame.frame.ignored', 'frame.frame.protocols', 'eth.eth.dst_raw[0]', 'eth.eth.dst_raw[1]', 'eth.eth.dst_raw[2]', 'eth.eth.dst_raw[3]', 'eth.eth.dst_raw[4]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst_resolved_raw[1]', 'eth.eth.dst_tree.eth.dst_resolved_raw[2]', 'eth.eth.dst_tree.eth.dst_resolved_raw[3]', 'eth.eth.dst_tree.eth.dst_resolved_raw[4]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[1]', 'eth.eth.dst_tree.eth.dst.oui_raw[2]', 'eth.eth.dst_tree.eth.dst.oui_raw[3]', 'eth.eth.dst_tree.eth.dst.oui_raw[4]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr_raw[1]', 'eth.e

In [16]:
df_nonan_converted.drop(columns=['frame.frame.time_utc','frame.frame.time','frame.frame.protocols'],inplace=True)


In [17]:

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Supponiamo che 'label' sia la colonna target e tutte le altre colonne siano features
X = df_nonan_converted.drop(columns=['label'])
y = df_nonan_converted['label']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)



In [18]:
# Definire il numero di feature da selezionare
k = 68

# Applicare SelectKBest con la funzione di score f_classif (ANOVA)
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X_scaled, y)

# Ottenere i punteggi delle features
scores = selector.scores_

# Ottenere le feature selezionate
selected_features = selector.get_support(indices=True)

# Ottenere i nomi delle colonne selezionate
selected_feature_names = X.columns[selected_features].tolist()

print("Le 68 features migliori selezionate dall'ANOVA sono:")
print(selected_feature_names)


# Se desideri memorizzarle in una variabile lista
features_list = selected_feature_names
print(features_list)



Le 68 features migliori selezionate dall'ANOVA sono:
['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  0   2   9  10  12  13  14  15  18  19  20  21  23  24  25  26  29  30
  31  32  35  36  37  38  40  41  42  43  46  47  48  49  52  53  54  55
  58  59  60  61  64  65  66  67  70  71  72  73  76  77  78  79  81  82
  83  84  87  88  89  90  92  93  94  95  98  99 100 101 103 104 105 106
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [19]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler

# Supponiamo che 'df' sia il tuo DataFrame e 'label' sia la colonna target
X = df_colonne_senza_nan.drop(columns=['label'])
y = df_colonne_senza_nan['label']

# Identifica le colonne costanti
constant_features = [col for col in X.columns if X[col].nunique() == 1]
print("Colonne costanti:")
print(constant_features)

# Rimuovi le colonne costanti
X = X.drop(columns=constant_features)

# Normalizza le features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definisci il numero di feature da selezionare
k = 68

# Applicare SelectKBest con ANOVA
selector = SelectKBest(score_func=f_classif, k=k)
X_new = selector.fit_transform(X_scaled, y)

# Ottenere gli indici delle feature selezionate
selected_features_indices = selector.get_support(indices=True)

# Ottenere i punteggi delle features
feature_scores = selector.scores_

# Ottenere i nomi delle colonne selezionate
selected_feature_names = X.columns[selected_features_indices].tolist()

# Creare un DataFrame per visualizzare i punteggi
scores_df = pd.DataFrame({
    'Feature': X.columns,
    'Score': feature_scores
})

# Ordinare i punteggi in ordine decrescente
scores_df = scores_df.sort_values(by='Score', ascending=False)

# Visualizzare i punteggi e i nomi delle features selezionate
print("Punteggi delle Features:")
print(scores_df)

print("\nLe 68 features migliori selezionate dall'ANOVA sono:")
print(selected_feature_names)



Colonne costanti:
['frame.frame.encap_type', 'frame.frame.offset_shift', 'frame.frame.marked', 'frame.frame.ignored', 'eth.eth.dst_raw[1]', 'eth.eth.dst_raw[2]', 'eth.eth.dst_raw[3]', 'eth.eth.dst_raw[4]', 'eth.eth.dst_tree.eth.dst_resolved_raw[1]', 'eth.eth.dst_tree.eth.dst_resolved_raw[2]', 'eth.eth.dst_tree.eth.dst_resolved_raw[3]', 'eth.eth.dst_tree.eth.dst_resolved_raw[4]', 'eth.eth.dst_tree.eth.dst.oui_raw[1]', 'eth.eth.dst_tree.eth.dst.oui_raw[2]', 'eth.eth.dst_tree.eth.dst.oui_raw[3]', 'eth.eth.dst_tree.eth.dst.oui_raw[4]', 'eth.eth.dst_tree.eth.addr_raw[1]', 'eth.eth.dst_tree.eth.addr_raw[2]', 'eth.eth.dst_tree.eth.addr_raw[3]', 'eth.eth.dst_tree.eth.addr_raw[4]', 'eth.eth.dst_tree.eth.addr_resolved_raw[1]', 'eth.eth.dst_tree.eth.addr_resolved_raw[2]', 'eth.eth.dst_tree.eth.addr_resolved_raw[3]', 'eth.eth.dst_tree.eth.addr_resolved_raw[4]', 'eth.eth.dst_tree.eth.addr.oui_raw[1]', 'eth.eth.dst_tree.eth.addr.oui_raw[2]', 'eth.eth.dst_tree.eth.addr.oui_raw[3]', 'eth.eth.dst_tree.

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=68 is greater than n_features=37. All the features will be returned.
  warnings.warn(


In [20]:
listel=selected_feature_names
print(listel)
print(len(listel))

['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.eth.src.oui_resolved_raw[0]', 'eth.eth.src_tree.eth.a

In [21]:
df_clean=df_clean[listel]
df_pf_1m_05h=df_pf_1m_05h[listel]

In [22]:
df_clean=fn.df_convert(df_clean,listel)
df_pf_1m_05h=fn.df_convert(df_pf_1m_05h,listel)

TypeError: df_convert() takes 1 positional argument but 2 were given

In [29]:
print(df_clean.dtypes)

frame.frame.time_epoch                           timedelta64[ns]
frame.frame.time_delta                                    object
frame.frame.time_delta_displayed                          object
frame.frame.time_relative                                 object
frame.frame.number                                        object
frame.frame.len                                           object
frame.frame.cap_len                                       object
eth.eth.dst_raw[0]                                        object
eth.eth.dst                                               object
eth.eth.dst_tree.eth.dst_resolved_raw[0]                  object
eth.eth.dst_tree.eth.dst.oui_raw[0]                       object
eth.eth.dst_tree.eth.dst.oui                              object
eth.eth.dst_tree.eth.addr_raw[0]                          object
eth.eth.dst_tree.eth.addr                                 object
eth.eth.dst_tree.eth.addr_resolved_raw[0]                 object
eth.eth.dst_tree.eth.addr

In [23]:

# Supponiamo che df_clean e df_pf_1m_05h siano i tuoi DataFrame

# Converti gli interi in pd.Timedelta, se necessario

df_clean['frame.frame.time_epoch'] = pd.to_timedelta(df_clean['frame.frame.time_epoch'], unit='s')
df_pf_1m_05h['frame.frame.time_epoch'] = pd.to_timedelta(df_pf_1m_05h['frame.frame.time_epoch'], unit='s')

In [27]:
# Calcola i cambiamenti di tempo
df_clean['capture_id'] = (df_clean['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=60)).cumsum()
df_pf_1m_05h['capture_id'] = (df_pf_1m_05h['frame.frame.time_epoch'].diff() > pd.Timedelta(seconds=60)).cumsum()


In [25]:
df_clean['label'] = 0
df_pf_1m_05h['label'] = 1

In [26]:
df_clean=fn.df_convert(df_clean, listel)
df_pf_1m_05h=fn.df_convert(df_pf_1m_05h, listel)

AttributeError: Can only use .str accessor with string values!

In [26]:
concat = pd.concat([df_clean, df_pf_1m_05h], axis=0)

In [27]:
print(concat.columns.tolist())

['frame.frame.time_epoch', 'frame.frame.time_delta', 'frame.frame.time_delta_displayed', 'frame.frame.time_relative', 'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len', 'eth.eth.dst_raw[0]', 'eth.eth.dst', 'eth.eth.dst_tree.eth.dst_resolved_raw[0]', 'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui', 'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr', 'eth.eth.dst_tree.eth.addr_resolved_raw[0]', 'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui', 'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg', 'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg', 'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig', 'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig', 'eth.eth.src_raw[0]', 'eth.eth.src', 'eth.eth.src_tree.eth.src_resolved_raw[0]', 'eth.eth.src_tree.eth.src.oui_raw[0]', 'eth.eth.src_tree.eth.src.oui', 'eth.eth.src_tree.eth.src.oui_resolved_raw[0]', 'eth.eth.src_tree.eth.a

In [28]:
concat['group_id']=concat.groupby(['capture_id', 'label']).ngroup()




In [29]:
fn.df_convert(concat,listel)

AttributeError: Can only use .str accessor with string values!

In [31]:
concat.columns

Index(['frame.frame.time_epoch', 'frame.frame.time_delta',
       'frame.frame.time_delta_displayed', 'frame.frame.time_relative',
       'frame.frame.number', 'frame.frame.len', 'frame.frame.cap_len',
       'eth.eth.dst_raw[0]', 'eth.eth.dst',
       'eth.eth.dst_tree.eth.dst_resolved_raw[0]',
       'eth.eth.dst_tree.eth.dst.oui_raw[0]', 'eth.eth.dst_tree.eth.dst.oui',
       'eth.eth.dst_tree.eth.addr_raw[0]', 'eth.eth.dst_tree.eth.addr',
       'eth.eth.dst_tree.eth.addr_resolved_raw[0]',
       'eth.eth.dst_tree.eth.addr.oui_raw[0]', 'eth.eth.dst_tree.eth.addr.oui',
       'eth.eth.dst_tree.eth.dst.lg_raw[0]', 'eth.eth.dst_tree.eth.dst.lg',
       'eth.eth.dst_tree.eth.lg_raw[0]', 'eth.eth.dst_tree.eth.lg',
       'eth.eth.dst_tree.eth.dst.ig_raw[0]', 'eth.eth.dst_tree.eth.dst.ig',
       'eth.eth.dst_tree.eth.ig_raw[0]', 'eth.eth.dst_tree.eth.ig',
       'eth.eth.src_raw[0]', 'eth.eth.src',
       'eth.eth.src_tree.eth.src_resolved_raw[0]',
       'eth.eth.src_tree.eth.src.oui_r

In [32]:
concat.dtypes

frame.frame.time_epoch                           timedelta64[ns]
frame.frame.time_delta                                    object
frame.frame.time_delta_displayed                          object
frame.frame.time_relative                                 object
frame.frame.number                                        object
frame.frame.len                                           object
frame.frame.cap_len                                       object
eth.eth.dst_raw[0]                                        object
eth.eth.dst                                               object
eth.eth.dst_tree.eth.dst_resolved_raw[0]                  object
eth.eth.dst_tree.eth.dst.oui_raw[0]                       object
eth.eth.dst_tree.eth.dst.oui                              object
eth.eth.dst_tree.eth.addr_raw[0]                          object
eth.eth.dst_tree.eth.addr                                 object
eth.eth.dst_tree.eth.addr_resolved_raw[0]                 object
eth.eth.dst_tree.eth.addr

In [29]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Supponendo che concat sia il DataFrame con i dati aggregati
# Separare le caratteristiche e le etichette
X = concat.drop(['label', 'capture_id'], axis=1)
y = concat['label']

# Verifica delle dimensioni
print("Dimensioni di X:", X.shape)
print("Dimensioni di y:", y.shape)

# Suddividere i dati in set di addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Normalizzare le caratteristiche
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Creare e addestrare il modello SVM
svm_model = SVC(kernel='rbf', C=10.0, gamma='scale')
svm_model.fit(X_train, y_train)

# Fare previsioni
y_pred = svm_model.predict(X_test)

# Stampa dei risultati
for i in range(len(y_test)):
    print(f"Elemento {i + 1}:")
    print(f"  Caratteristiche: {X_test[i]}")
    print(f"  Classe prevista: {y_pred[i]}")
    print(f"  Classe effettiva: {y_test.values[i]}\n")

# Stampa della Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Stampa del Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Dimensioni di X: (101842, 38)
Dimensioni di y: (101842,)


TypeError: float() argument must be a string or a real number, not 'Timedelta'